B1: Cài và import thư viện cần thiết

In [1]:
import web3
import solcx
import json

In [2]:
# kiểm tra phiên bản web3
print('Web3 Version', web3.__version__)

Web3 Version 7.10.0


In [3]:
#Kiêm rtra chức năng của solcx
print("Các phiên bản Solidity có sẵn:",solcx.get_installed_solc_versions())

Các phiên bản Solidity có sẵn: [<Version('0.8.0')>]


In [4]:
print("Các phiên bản Solidity có thể cài:",solcx.get_installable_solc_versions())

Các phiên bản Solidity có thể cài: [<Version('0.8.29')>, <Version('0.8.28')>, <Version('0.8.27')>, <Version('0.8.26')>, <Version('0.8.25')>, <Version('0.8.24')>, <Version('0.8.23')>, <Version('0.8.22')>, <Version('0.8.21')>, <Version('0.8.20')>, <Version('0.8.19')>, <Version('0.8.18')>, <Version('0.8.17')>, <Version('0.8.16')>, <Version('0.8.15')>, <Version('0.8.14')>, <Version('0.8.13')>, <Version('0.8.12')>, <Version('0.8.11')>, <Version('0.8.10')>, <Version('0.8.9')>, <Version('0.8.8')>, <Version('0.8.7')>, <Version('0.8.6')>, <Version('0.8.5')>, <Version('0.8.4')>, <Version('0.8.3')>, <Version('0.8.2')>, <Version('0.8.1')>, <Version('0.8.0')>, <Version('0.7.6')>, <Version('0.7.5')>, <Version('0.7.4')>, <Version('0.7.3')>, <Version('0.7.2')>, <Version('0.7.1')>, <Version('0.7.0')>, <Version('0.6.12')>, <Version('0.6.11')>, <Version('0.6.10')>, <Version('0.6.9')>, <Version('0.6.8')>, <Version('0.6.7')>, <Version('0.6.6')>, <Version('0.6.5')>, <Version('0.6.4')>, <Version('0.6.3')>, <

B2: Cài đặt Solidity Cmpiler 0.8.0

In [5]:
from solcx import install_solc,compile_standard
install_solc('0.8.0')

<Version('0.8.0')>

In [6]:
print('Phiên bản đã cài',solcx.get_installed_solc_versions())

Phiên bản đã cài [<Version('0.8.0')>]


B3

In [7]:
with open("Message.sol","r") as file:
    source_code=file.read()

#Kiểm tra xem đọc đúng chưa
print(source_code[:200])

// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

contract MessageContract {
    string public message;

    constructor(string memory _message) {
        message= _message;
    }
    function 


In [8]:
compiled_sol = compile_standard({
    "language": "Solidity",
    "sources": {
        "Message.sol": {
            "content": source_code
        }
    },
    "settings": {
        "outputSelection": {
            "*": {
                "*": ["abi", "metadata", "evm.bytecode", "evm.sourceMap"]
            }
        }
    }
}, solc_version="0.8.0")


3.3 - Trích xuất ABI và Bytecode

In [9]:
abi=compiled_sol['contracts']['Message.sol']['MessageContract']['abi']
bycode=compiled_sol['contracts']['Message.sol']['MessageContract']['evm']['bytecode']['object']

In [11]:
print("ABI rút gọn", abi[:1])
print('Bycode (rút gọn)', bycode[:20],"...")

ABI rút gọn [{'inputs': [{'internalType': 'string', 'name': '_message', 'type': 'string'}], 'stateMutability': 'nonpayable', 'type': 'constructor'}]
Bycode (rút gọn) 60806040523480156100 ...


Bước 4 - Kết nối ganache và triển khai hợp đồng 

Mục tiêu: 

1. Kết nối với Ganache đang chạy tại http://127.0.0.1:7545
2. Triển khai MessageContract vơi constructor nhận chuỗi "Chào Anh/Chị"

In [ ]:
# from web3 import Web3

# # Kết nối tới Ganache
# w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:7545"))

# # Kiểm tra kết nối
# print("Connected?", w3.is_connected())

# # Lấy danh sách tài khoản
# accounts = w3.eth.accounts

# # Thiết lập tài khoản mặc định
# w3.eth.default_account = accounts[0]

# # Gửi ETH từ account[0] -> account[1]
# tx_hash = w3.eth.send_transaction({
#     'from': accounts[0],
#     'to': accounts[1],
#     'value': w3.to_wei(1, 'ether')  # gửi 1 ETH
# })

# # In hash giao dịch
# print("TX Hash:", tx_hash.hex())

# # Đợi transaction được khai thác
# tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
# print("Giao dịch hoàn tất tại block:", tx_receipt.blockNumber)


Connected? True
TX Hash: 351f0609b2a5e3ec029b5b1ddc01b2757bd21d85a7eac251d34b5d0793a8147e
Giao dịch hoàn tất tại block: 1


In [25]:
from web3 import Web3
# Kết nối tới Ganache GUI
w3=Web3(Web3.HTTPProvider("http://127.0.0.1:7545"))

#Kiểm tra kết nối
print("Kết nối thành công ? --> ",w3.is_connected())

#Lấy danh sách tài khoản có sẵn trong Ganache
# accounts=w3.eth.accounts[0]
# w3.eth.default_account=accounts[0]
w3.eth.default_account='0x3AAA39605341726FFD91F979E659f8928A9d6675'
print("Tài khoản mặc định:",w3.eth.default_account)

Kết nối thành công ? -->  True
Tài khoản mặc định: 0x3AAA39605341726FFD91F979E659f8928A9d6675


In [26]:
#Tạo đối tượng contract từ ABI và Bycode
MessageContract=w3.eth.contract(abi=abi,bytecode=bycode)

#Gửi giao dịch triển khai contract, truyền constructor message
tx_hash=MessageContract.constructor("Chào quý khách!").transact()

#Chờ nhận kết quả giao dịch để lấy địa chỉ contract
tx_receipt=w3.eth.wait_for_transaction_receipt(tx_hash)

#In địa chỉ contract vừa triển khai
contract_address=tx_receipt.contractAddress
print('Hợp đồng đã triển khai tại địa chỉ:',contract_address)

Hợp đồng đã triển khai tại địa chỉ: 0xB31F81b7e07A110f65606b0f10558f803353911D


B5: Tương tác với smart contract

5.1. Tạo đối tượng contract để tương tác

In [27]:
# Tạo instance của contract tại địa chỉ đã triển khai
message_contract_instance=w3.eth.contract(
    address=contract_address,
    abi=abi
)

In [28]:
contract_address

'0xB31F81b7e07A110f65606b0f10558f803353911D'

In [32]:
# Khởi tạo đúng contract
MessageContract = w3.eth.contract(address=contract_address, abi=abi)

# Đặt tài khoản mặc định
w3.eth.default_account = w3.eth.accounts[0]

# Gửi transaction
tx_hash = MessageContract.functions.updateMessage("CT đã được cập nhật").transact()
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

print("Cập nhật thành công tại block:", tx_receipt.blockNumber)

Cập nhật thành công tại block: 3


5.2. Đọc thông điệp hiện tại 

In [33]:
current_message=message_contract_instance.functions.message().call()
print("Thông điệp hiện tại tron contract:",current_message)

Thông điệp hiện tại tron contract: CT đã được cập nhật
